In [1]:
# 讀資料庫 >> df
# 讀exp 資料
import pandas as pd
import pymysql

db_settings = {
        "host": "",
        "port": ,
        "user": "",
        "password": "",
        "db": "",
        "charset": "utf8"
        }

db_timeformat = "%Y-%m-%d %H:%M:%S"
conn = pymysql.connect(**db_settings)

with conn.cursor() as cursor:
    sql = "SELECT * FROM `ptt_content` WHERE `tp` BETWEEN '2021-07-01' AND '2021-07-10'"
    cursor.execute(sql)
    r = cursor.fetchall()
    content_df = pd.DataFrame(list(r),
                              columns=['url', 'title', 'content', 'time', 'author', 'board', 'ip', 'country', 'push_count'])

    print('1. content done')
    sql = "SELECT * FROM `ptt_push` WHERE `tp` BETWEEN '2021-07-01' AND '2021-07-10'"
    cursor.execute(sql)
    r = cursor.fetchall()
    push_df = pd.DataFrame(list(r),
                           columns=['url', 'seq', 'board', 'tag', 'content', 'time', 'userid', 'reply'])
    print('2. push done')
    
with conn.cursor() as cursor:
    sql = "SELECT * FROM `news` WHERE `published_time` BETWEEN '2021-07-01' AND '2021-07-10'"
    cursor.execute(sql)
    r = cursor.fetchall()
    news_df = pd.DataFrame(list(r),
                              columns=['url', 'title', 'content', 'author', 'board', 'tag','source','published_time','collected_time'])


1. content done
2. push done


In [4]:
# 讀ptt訓練資料
db_settings = {
        "host": "localhost",
        "port": 3306,
        "user": "root",
        "password": "ai52004800",
        "db": "lab",
        "charset": "utf8"
        }

db_timeformat = "%Y-%m-%d %H:%M:%S"
conn = pymysql.connect(**db_settings)

with conn.cursor() as cursor:
    sql = "SELECT * FROM `ptt_content` WHERE `tp` BETWEEN '2021-01-01' AND '2021-07-01'"
    cursor.execute(sql)
    r = cursor.fetchall()
    train_content_df = pd.DataFrame(list(r),
                              columns=['url', 'title', 'content', 'time', 'author', 'board', 'ip', 'country', 'push_count'])
    print('1. train_content_df done')
    

1. train_content_df done


In [2]:
import warnings
warnings.filterwarnings('ignore') 
import pickle
import jieba
import json
from tqdm.notebook import tqdm
from os import listdir

In [3]:
for f in listdir("./predefined/dictionary/"):
    jieba.load_userdict("./predefined/dictionary/"+f)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\AICENT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.771 seconds.
Prefix dict has been built successfully.


In [57]:
stop_wd = set()
with open("./predefined/stopwords_all.json", 'r', encoding='utf8') as f:
    for w in json.load(f):
        stop_wd.add(w)
        
with open("./predefined/stopwords_news.json", 'r', encoding='utf8') as f:
    for w in json.load(f):
        stop_wd.add(w)

#### filter stopwords

In [60]:
article_data = pd.concat([train_content_df.content, news_df.content]).reset_index(drop=True)
# 斷詞轉換寫檔
with open('article_data_text.txt', 'w', encoding='utf-8') as f:
    for article in tqdm(article_data):

        article = article.strip()
        data = jieba.cut(article)
        data = [word for word in data if word != ' ' and  word not in stop_wd]
        data = ' '.join(data)
            
        f.write(data+'\n')

  0%|          | 0/239094 [00:00<?, ?it/s]

In [1]:
# from gensim.corpora import WikiCorpus
# # https://clay-atlas.com/blog/2020/01/17/python-chinese-tutorial-gensim-word2vec/
# # wiki輔助訓練word2vec 模型
# wiki_corpus = WikiCorpus(r'C:\Users\AICenterM365Lic\Downloads\zhwiki-20210701-pages-articles-multistream.xml.bz2', dictionary={})
# text_num = 0

# with open('wiki_text.txt', 'w', encoding='utf-8') as f:
#     for text in wiki_corpus.get_texts():
#         f.write(' '.join(text)+'\n')
#         text_num += 1
#         if text_num % 10000 == 0:
#             print('{} articles processed.'.format(text_num))

#     print('{} articles processed.'.format(text_num))

In [62]:
from opencc import OpenCC
# Initial
cc = OpenCC('s2t')
# Tokenize
with open('article_data_text.txt', 'a', encoding='utf-8') as new_f:
    with open('wiki_text.txt', 'r', encoding='utf-8') as f:
        for data in tqdm(f):
            data = cc.convert(data)
            data = jieba.cut(data)
            data = [word for word in data if word != ' ' and word not in stop_wd]
            data = ' '.join(data)

            new_f.write(data+'\n')

0it [00:00, ?it/s]

### w2v訓練

In [37]:
# # save corpus_data
# with open("./corpus.txt", 'w') as f:
#     for i in new_corpus:
#         f.write(str(i)+"\n")

In [39]:
# # 讀進wiki資料
# with open('./predefined/dictionary/ptt.txt', 'r', encoding='utf-8') as new_f:
#     wiki_data = [f for f in new_f.read()]
    
# wiki_data[:10]

In [63]:
# 主要透過 gensim 訓練成 model 並供使用
class Train(object):

    def __init__(self):
        pass

    # 可參考 https://radimrehurek.com/gensim/models/word2vec.html 更多運用
    def train(self):
        print("訓練中...")
        # Load file
        sentence = word2vec.LineSentence("article_data_text.txt")
        # Setting degree and Produce Model(Train)
        # Settings
        seed = 666
        sg = 0
        window_size = 10
        vector_size = 100
        min_count = 1
        workers = 8
        epochs = 5
        batch_words = 10000
        
        model = word2vec.Word2Vec(
                sentence,
                min_count=min_count,
                vector_size=vector_size,
                workers=workers,
                epochs=epochs,
                window=window_size,
                sg=sg,
                seed=seed,
                batch_words=batch_words
                )
        # Save model 
        model.save('word2vec.model')
        model.wv.save_word2vec_format(u"w2v.model.bin", binary = True)
        print("model 已儲存完畢")

if __name__ == "__main__":
    t = Train()
    # 訓練(shallow semantic space)
    t.train()

訓練中...
model 已儲存完畢


In [64]:
from gensim.models.keyedvectors import KeyedVectors
# How to use bin(model)?
word_vectors = KeyedVectors.load_word2vec_format("w2v.model.bin", binary = True)

In [73]:
print("外文 前10名相似:")    
res = word_vectors.most_similar('外文', topn = 20)
for item in res:
    print(item[0] + "," + str(item[1]))

外文 前10名相似:
西文,0.7729334831237793
中英文,0.6763495206832886
中文書籍,0.6715853214263916
o9NO7D,0.664551317691803
譯自,0.6533045172691345
中文,0.652868926525116
中文期刊,0.6455404162406921
英語翻譯,0.6444995403289795
譯寫,0.638895571231842
翻譯,0.6315523982048035
筆譯,0.6294631958007812
刊名,0.6286423802375793
英文翻譯,0.6213029623031616
簡體,0.6150707006454468
專業書籍,0.6088368892669678
譯版,0.6057499051094055
察布查爾報,0.6038843989372253
中文名,0.6009138822555542
英漢對照,0.6000564098358154
對譯,0.5964251160621643


In [23]:
# model.save('word2vec.model')